# Ensemble

Sử dụng kỹ thuật Model Ensemble để tăng cường độ chính xác khi suy diễn

In [6]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import glob
import cv2
import torch.nn.functional as F
from torch.autograd import Variable
import os

import torchvision
import torchvision.transforms as transforms

from torch.nn import CrossEntropyLoss, Dropout, Softmax, Linear, Conv2d, LayerNorm
import matplotlib.pyplot as plt
from torchsummary import summary

In [9]:
def load_data(data_dir="./data"):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset = torchvision.datasets.CIFAR10(
        root=data_dir, train = True, download = True, transform = transform)

    testset = torchvision.datasets.CIFAR10(
        root=data_dir, train = False, download = True, transform = transform)

    return trainset, testset


class Net(nn.Module):
    def __init__(self, l1 = 120, l2 = 84):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.fc3 = nn.Linear(l2, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

model = Net()
if torch.cuda.is_available():
    model.cuda()
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             456
         MaxPool2d-2            [-1, 6, 14, 14]               0
            Conv2d-3           [-1, 16, 10, 10]           2,416
         MaxPool2d-4             [-1, 16, 5, 5]               0
            Linear-5                  [-1, 120]          48,120
            Linear-6                   [-1, 84]          10,164
            Linear-7                   [-1, 10]             850
Total params: 62,006
Trainable params: 62,006
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.06
Params size (MB): 0.24
Estimated Total Size (MB): 0.31
----------------------------------------------------------------


In [10]:
# Accuracy on test set

def test_accuracy(net, device = "cpu"):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

In [11]:
def train(net, criterion, optimizer, save_path, device = "cpu"):
    T_cur = 0
    for epoch in range(1, epochs + 1):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        T_cur += 1
        
        # warm-up
        if epoch <= warm_epoch:
            optimizer.param_groups[0]['lr'] = (1.0 * epoch) / warm_epoch  * init_lr
        else: 
            # cosine annealing lr
            optimizer.param_groups[0]['lr'] = last_lr + (init_lr - last_lr) * (1 + np.cos(T_cur * np.pi / T_max)) / 2

        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if i + 1 == len(trainloader):
                print("[Epoch %d] loss: %.3f" % (epoch, running_loss / epoch_steps))
                running_loss = 0.0
                
    print("Finished Training")
    print("Test accuracy:", test_accuracy(net, device))
    torch.save(net.state_dict(), save_path)

In [ ]:
epochs = 10
warm_epoch = 5
init_lr = 1e-2
last_lr = 1e-4
T_max = epochs

configs = [{'l1': 64, 'l2': 32}, {'l1': 128, 'l2': 64}]

trainset, testset = load_data('./data')
trainloader = torch.utils.data.DataLoader(
    trainset,
    batch_siz = 128,
    shuffle = True,
)
testloader = torch.utils.data.DataLoader(
    testset, batch_size = 4, shuffle = False, num_workers = 2)

100%|██████████| 170498071/170498071 [39:35<00:00, 71784.99it/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [13]:
# huấn luyện hai model trong config

os.makedirs('./snapshot', exist_ok = True)

for i, cfg in enumerate(configs):
    print(cfg)
    net = Net(cfg['l1'], cfg['l2'])
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr = init_lr, momentum = 0.9)

    save_path = f'./snapshot/model{i}.pth'
    train(net, criterion, optimizer, save_path, device)

{'l1': 64, 'l2': 32}
[Epoch 1] loss: 2.302
[Epoch 2] loss: 2.111
[Epoch 3] loss: 1.718
[Epoch 4] loss: 1.517
[Epoch 5] loss: 1.380
[Epoch 6] loss: 1.224
[Epoch 7] loss: 1.170
[Epoch 8] loss: 1.135
[Epoch 9] loss: 1.114
[Epoch 10] loss: 1.107
Finished Training
Test accuracy: 0.5889
{'l1': 128, 'l2': 64}
[Epoch 1] loss: 2.301
[Epoch 2] loss: 2.133
[Epoch 3] loss: 1.724
[Epoch 4] loss: 1.483
[Epoch 5] loss: 1.358
[Epoch 6] loss: 1.214
[Epoch 7] loss: 1.156
[Epoch 8] loss: 1.117
[Epoch 9] loss: 1.094
[Epoch 10] loss: 1.086
Finished Training
Test accuracy: 0.592


In [ ]:
# kết hợp hai models

from tqdm import tqdm

def test_ensemble(device = "cuda:0"):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in tqdm(testloader):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            final_outputs = torch.zeros((4, 10)).to(device)
            for i, cfg in enumerate(configs):
                net = Net(cfg['l1'], cfg['l2'])
                net.to(device) 
                net.load_state_dict(torch.load(f'./snapshot/model{i}.pth'))               
                outputs = net(images)
                final_outputs = final_outputs.add(outputs)

            final_outputs.div(len(configs))
            _, predicted = torch.max(final_outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

In [17]:
test_ensemble(device = device)

100%|██████████| 2500/2500 [00:20<00:00, 119.55it/s]


0.6137